In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%reload_ext autoreload
%autoreload 2
import itertools
import sys
sys.path.append('/home/roquero/CausalAggregation/Code')
from generateEnvironment import GenerateEnvironment
from robust_estimator import SolveProblem
import matplotlib as mpl
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
np.set_printoptions(suppress=True)
np.set_printoptions(precision=5)

/home/roquero/miniconda3/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
connectivity_e0 = np.array([
    [0,0,0,0,0,0,0,0], # H / 0
    [0,0,0,0,0,0,0,0], # I / 1
    [2,0,0,0,-1,0,0,0], # X_1 / 2
    [1,0,1,0,0,0,0,0], # X_2 / 3
    [0,0,0,2,0,0,0,0], # X_3 / 4
    [0,0,1,0,1,0,0,0], # X_4 / 5
    [1,0,0,1,0,2,0,0], # Y / 6
    [0,0,0,1,0,2,-1,0], # X_5 / 7
])
x_indices=np.array([2,3,4,5,7])
y_index=6
n_samples=50000

In [3]:
interventional_indices = [2,7]
solver = SolveProblem(connectivity_e0, x_indices, y_index)

list_dict_interventions = [{i:{'type':'independent'} for i in np.array(interventional_indices)}]

list_environments = [solver.generate_intervention(n_samples, dict_interventions)
                     for dict_interventions in list_dict_interventions]

In [4]:
M, _ = solver.combine_constraints(list_environments)
delta = solver.compute_wass_radius(solver.generate_intervention(n_samples, {}), M, interventional_indices)

In [5]:
np.sqrt(delta)

6.726764497922964

In [6]:
problem = solver.robust_estimator(np.sqrt(delta), list_environments, solver.generate_intervention(n_samples, {}))
beta_hat = problem.variables()[0].value

## Min max upper bound

In [7]:
problem.value**2

1.509472252554586

In [8]:
solver = SolveProblem(connectivity_e0,x_indices,y_index)
interventional_environment = solver.generate_intervention(n_samples,
                        {i:{'type':'do-zero'} for i in np.array(interventional_indices)})['dataset']

solver = SolveProblem(connectivity_e0,x_indices,y_index)
observational_environment = solver.generate_intervention(n_samples,{})['dataset']

lr = LinearRegression()
_  = lr.fit(observational_environment[x_indices,:].T, observational_environment[y_index,:])
beta_LR_obs = lr.coef_

lr = LinearRegression()
_  = lr.fit(interventional_environment[x_indices,:].T, interventional_environment[y_index,:])
beta_LR_int = lr.coef_

In [9]:
solver.beta
beta_hat
beta_LR_obs
beta_LR_int

array([0, 1, 0, 2, 0])

array([-0.50085,  1.50325,  0.00459,  1.99848,  0.00472])

array([ 0.08108,  1.06816,  0.15797,  2.00244, -0.53883])

array([0.     , 1.4941 , 0.00644, 1.99831, 0.     ])

## Performance over the interventional do-zero dataset.

Robust estimator

In [10]:
np.sum(np.square(interventional_environment[x_indices,:].T.dot(beta_hat)
               -interventional_environment[y_index,:]))/interventional_environment.shape[1]

1.4989638104582685

Beta^0 estimator

In [11]:
np.sum(np.square(interventional_environment[x_indices,:].T.dot(solver.beta)
               -interventional_environment[y_index,:]))/interventional_environment.shape[1]

2.002609446202562

beta_OLS on the interventional dataset

In [12]:
np.sum(np.square(interventional_environment[x_indices,:].T.dot(beta_LR_int)
               -interventional_environment[y_index,:]))/interventional_environment.shape[1]

1.4988942656863666

beta_OLS on observational dataset

In [13]:
np.sum(np.square(interventional_environment[x_indices,:].T.dot(beta_LR_obs)
               -interventional_environment[y_index,:]))/interventional_environment.shape[1]

1.5499299948846217

## Performance over observational dataset.

Robust estimator

In [14]:
np.sum(np.square(observational_environment[x_indices,:].T.dot(beta_hat)
               -observational_environment[y_index,:]))/observational_environment.shape[1]

1.5245971644874359

Beta^0 estimator

In [15]:
np.sum(np.square(observational_environment[x_indices,:].T.dot(solver.beta)
               -observational_environment[y_index,:]))/observational_environment.shape[1]

2.004602663438213

beta_OLS on observational dataset

In [16]:
np.sum(np.square(observational_environment[x_indices,:].T.dot(beta_LR_obs)
               -observational_environment[y_index,:]))/observational_environment.shape[1]

0.5416097936044657

beta_OLS on the interventional dataset

In [17]:
np.sum(np.square(observational_environment[x_indices,:].T.dot(beta_LR_int)
               -observational_environment[y_index,:]))/observational_environment.shape[1]

1.341212033756126

## Additional sanity checks